### 주차별 프로젝트(1주차 : 종이 헬리콥터 회귀분석)

구성 변수 : 날개 길이, 날개 폭, 다리 길이, 몸통 길이 + ɑ

모델 구성 : scikit-learn, NN

### 데이터 구성
실제로 만들어서 시간을 관측하고, 시간을 측정하는 기준은 아래와 같다.

- 두 날개의 가장 안쪽을 두 손가락을 이용해 천장에 붙인 뒤 낙하

In [18]:
import pandas as pd

dataset = {
    "wing_length" : [11.0, 9.5, 10.2, 16.0, 9.5, 13.25, 13.25, 11.8, 8.5, 8.0, 9.7, 9.0, 9.0, 9.0, 7.5],
    "wing_width" : [6.25, 7.4, 5.3, 7.4, 7.4, 7.45, 7.45, 6.95, 2.7, 2.7, 3.0, 3.0, 3.0, 3.0, 2.7],
    "body_length" : [3.0, 3.0, 1.8, 1.0, 3.0, 2.0, 2.0, 2.25, 2.0, 2.0, 3.0, 3.0, 2.5, 2.15, 1.65],
    "leg_length" : [7.05, 8.5, 2.5, 4.0, 3.6, 4.65, 3.8, 4.8, 2.5, 2.5, 3.5, 3.5, 4.0, 4.3, 2.8],
    "target" : [2.31, 1.77, 2.73, 2.43, 2.27, 2.5, 2.28, 2.41, 2.41, 2.76, 2.98, 2.69, 2.73, 2.8, 2.83]
}

len(dataset['wing_length'])
df = pd.DataFrame(dataset, columns = dataset.keys(), index = list(range(len(dataset['wing_length']))))
df.shape

(15, 5)

### 학습 데이터 구성

In [19]:
X = df.drop(labels = 'target', axis = 1)
y = df[['target']]

In [20]:
X

,wing_length,wing_width,body_length,leg_length
0,11.00,6.25,3.00,7.05
1,9.50,7.40,3.00,8.50
2,10.20,5.30,1.80,2.50
3,16.00,7.40,1.00,4.00
4,9.50,7.40,3.00,3.60
5,13.25,7.45,2.00,4.65
6,13.25,7.45,2.00,3.80
7,11.80,6.95,2.25,4.80
8,8.50,2.70,2.00,2.50
9,8.00,2.70,2.00,2.50


In [21]:
y

,target
0,2.31
1,1.77
2,2.73
3,2.43
4,2.27
5,2.50
6,2.28
7,2.41
8,2.41
9,2.76


### 모델 빌드
- input layer's shape = (4, ) ➡️ 4개의 독립변수
- 64개의 노드를 가지는 hidden layer 4개층 구성, 활성화 함수는 relu 사용
- output layer's shape = (1, ) ➡️ 비행시간에 해당하는 1개의 출력 변수, 항등함수로 활성화

### scikit learn
다중 선형회귀 분석 모델들로 시도

In [22]:
from keras.models import Model
from keras.layers import Dense, Input
from keras.optimizers import RMSprop, Adam

input_layer = Input(shape = (4, ))
x = Dense(64, activation = "relu")(input_layer)
x = Dense(64, activation = 'relu')(x)
x = Dense(64, activation = 'relu')(x)
x = Dense(64, activation = 'relu')(x)
output_layer = Dense(1)(x)

optim = Adam(learning_rate = 0.001)

model = Model(input_layer, output_layer)
model.compile(optimizer = optim, loss = "mean_squared_error")

In [23]:
model.fit(X, y, epochs = 500)

Epoch 1/500
1/1 [==============================] - 0s 242ms/step - loss: 7.9484
Epoch 2/500
1/1 [==============================] - 0s 3ms/step - loss: 5.9532
Epoch 3/500
1/1 [==============================] - 0s 5ms/step - loss: 4.3790
Epoch 4/500
1/1 [==============================] - 0s 8ms/step - loss: 3.1781
Epoch 5/500
1/1 [==============================] - 0s 5ms/step - loss: 2.2578
Epoch 6/500
1/1 [==============================] - 0s 6ms/step - loss: 1.5686
Epoch 7/500
1/1 [==============================] - 0s 5ms/step - loss: 1.0700
Epoch 8/500
1/1 [==============================] - 0s 5ms/step - loss: 0.7474
Epoch 9/500
1/1 [==============================] - 0s 5ms/step - loss: 0.6074
Epoch 10/500
1/1 [==============================] - 0s 5ms/step - loss: 0.6063
Epoch 11/500
1/1 [==============================] - 0s 5ms/step - loss: 0.6946
Epoch 12/500
1/1 [==============================] - 0s 6ms/step - loss: 0.8118
Epoch 13/500
1/1 [==============================] - 0s 5ms/

In [24]:
predicted_list = []
for i in range(X.shape[0]):
    predicted_list.append(model.predict([list(X.iloc[i, :].values)])[0][0])

1/1 [==============================] - 0s 28ms/step


In [25]:
df['predicted_target'] = predicted_list

In [26]:
df

,wing_length,wing_width,body_length,leg_length,target,predicted_target
0,11.00,6.25,3.00,7.05,2.31,2.306652
1,9.50,7.40,3.00,8.50,1.77,1.765457
2,10.20,5.30,1.80,2.50,2.73,2.717427
3,16.00,7.40,1.00,4.00,2.43,2.433959
4,9.50,7.40,3.00,3.60,2.27,2.263833
5,13.25,7.45,2.00,4.65,2.50,2.433766
6,13.25,7.45,2.00,3.80,2.28,2.333108
7,11.80,6.95,2.25,4.80,2.41,2.441673
8,8.50,2.70,2.00,2.50,2.41,2.554198
9,8.00,2.70,2.00,2.50,2.76,2.613729


In [35]:
import numpy as np

random_data = np.random.rand()

model.predict()

1/1 [==============================] - 0s 32ms/step


array([[3.0059345]], dtype=float32)